In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os, sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
pd.options.display.float_format = '{:,.4f}'.format

In [3]:
#aggregate diff experiments eg. top global (10 runs) + top local (10 runs)
def aggregated_logs_to_df(path: str):
    #Creates and returns a Pandas DataFrame from all logs in `path`
    experiments = [os.path.join(dp, f) for dp, dn, filenames
                   in os.walk(path) for f in filenames
                   if os.path.splitext(f)[1] == '.csv']
    experiments = {str(e): e for e in experiments}

    aggregate_df = pd.DataFrame()
    for exp_name, exp_path in experiments.items():
        if 'full_results' in exp_name:
            continue
        df = pd.read_csv(exp_path)
        df['exp'] = exp_name
        aggregate_df = pd.concat([aggregate_df, df])

    # write to file
    #aggregate_df.to_csv(os.path.join(path, 'summary.csv'), index=False)
    #aggregate_df.to_csv('experiments/summary.csv', index=False)
    return aggregate_df

In [4]:
#aggregate diff experiments eg. top global (10 runs) + top local (10 runs)
def full_logs_to_df(path: str):
    #Creates and returns a Pandas DataFrame from all logs in `path`
    experiments = [os.path.join(dp, f) for dp, dn, filenames
                   in os.walk(path) for f in filenames
                   if os.path.splitext(f)[1] == '.csv']
    experiments = {str(e): e for e in experiments}

    full_df = pd.DataFrame()
    for exp_name, exp_path in experiments.items():
        if 'aggregate_log' in exp_name:
            continue
        df = pd.read_csv(exp_path)
        df['exp'] = exp_name
        full_df = pd.concat([full_df, df])

    # write to file
    #aggregate_df.to_csv(os.path.join(path, 'summary.csv'), index=False)
    #aggregate_df.to_csv('experiments/summary.csv')
    return full_df

In [5]:
def manipulate_df(df, metric):
    df = df[['tag', 'value']].groupby(['tag'], as_index=True)
    df = df.agg(['mean', 'std'])
    df.columns = df.columns.droplevel(0)
    df = df.loc[metric]
    df.index = df.index.str.replace('eval/', '', regex=True)
    df.index = df.index.str.replace('learner/', '', regex=True)
    return df

In [6]:
def return_df(return_path, return_name, paths, keys, criteria):
    prepaired_df = []
    for path in paths:
        df = aggregated_logs_to_df(path=path)
        df = manipulate_df(df, criteria)
        prepaired_df.append(df)
    prepaired_df = pd.concat(prepaired_df, keys=keys, names=['experiment', 'criteria'])
    #prepaired_df.to_csv(os.path.join(return_path, return_name), index=False)
    return prepaired_df

In [7]:
def time_per_step_df(full_df):
    df = full_df.loc[full_df['tag'] == 'learner/time_per_step']
    df = df[['tag', 'value']].groupby(['tag'], as_index=False)
    df = df.agg(['mean', 'std'], as_index=False)
    df.index = df.index.str.replace('learner/', '', regex=True)
    df.columns = df.columns.droplevel(0)
    return df

In [8]:
def return_time_per_step_df(return_path, return_name, paths, keys):
    prepaired_df = []
    for path in paths:
        df = full_logs_to_df(path=path)
        df = time_per_step_df(df)
        prepaired_df.append(df)
    prepaired_df = pd.concat(prepaired_df, keys=keys, names=['experiment', 'criteria'])
    #prepaired_df.to_csv(os.path.join(return_path, return_name), index=False)
    return prepaired_df

In [9]:
def manipulate_full_df(full_df, metric, agg):
    df = full_df.loc[full_df['tag'] == metric]
    df = df[['tag', 'epoch', 'value']].groupby(['tag', 'epoch'], as_index=False)
    df = df.agg(agg, as_index=False)
    df.columns = df.columns.droplevel(0)
    df.index = df.index.droplevel(0)
    return df

In [10]:
def return_full_df(return_path, return_name, paths, keys, criteria):
    prepaired_df = []
    for path in paths:
        df = full_logs_to_df(path=path)
        df = manipulate_df(df, criteria)
        prepaired_df.append(df)
    prepaired_df = pd.concat(prepaired_df, keys=keys, names=['experiment', 'criteria'])
    #prepaired_df.to_csv(os.path.join(return_path, return_name), index=False)
    return prepaired_df

In [27]:
  #to calculate the epsilon (or any aggregated value) + mean time from full 
return_path = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/'
return_name = 'summary.csv'
path1 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-1-01'
path2 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-08-01'
path3 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-07-01'
path4 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-06-01'
path5 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-1-02'
path6 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-08-02'
path7 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-07-02'
path8 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-w-06-02'
path9 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-w-05-02'
path10 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-w-08-02-c'
path11 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-w-07-02-c'
path12 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-w-06-02-c'
path13 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-w-05-02-c'
path14 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-standard'
paths = [path1, path2, path3, path4, path5, path6, path7, path8, path9, path10, path11, path12, path13, path14]
keys = ['w = 1.0-0.1, c = 0.8', 'w = 0.8-0.1, c = 0.8', 'w = 0.7-0.1, c = 0.8', 'w = 0.6-0.1, c = 0.8', 'w = 1.0-0.2, c = 0.8', 'w = 0.8-0.2, c = 0.8', 'w = 0.7-0.2, c = 0.8', 'w = 0.6-0.2, c = 0.8', 'w = 0.5-0.2, c = 0.8', 'w = 0.8-0.2-c = 1.5', 'w = 0.7-0.2, c = 1.5', 'w = 0.6-0.2, c = 1.5', 'w = 0.5-0.2, c = 1.5', 'w fix']
criteria = ['eval/epsilon_relative']

df = return_df(return_path, return_name, paths, keys, criteria)
df

,,mean,std
experiment,criteria,,
"w = 1.0-0.1, c = 0.8",epsilon_relative,0.0023,0.0026
"w = 0.8-0.1, c = 0.8",epsilon_relative,0.0009,0.0007
"w = 0.7-0.1, c = 0.8",epsilon_relative,0.0028,0.0061
"w = 0.6-0.1, c = 0.8",epsilon_relative,0.0003,0.0003
"w = 1.0-0.2, c = 0.8",epsilon_relative,0.0016,0.0022
"w = 0.8-0.2, c = 0.8",epsilon_relative,0.0013,0.0016
"w = 0.7-0.2, c = 0.8",epsilon_relative,0.0008,0.0012
"w = 0.6-0.2, c = 0.8",epsilon_relative,0.0007,0.0009
"w = 0.5-0.2, c = 0.8",epsilon_relative,0.0003,0.0002


In [22]:
  #to calculate the epsilon (or any aggregated value) + mean time from full 
return_path = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/'
return_name = 'summary.csv'
path1 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-p02-g30'
path2 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-08-p04-g30'
path3 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p06-g30'
path4 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p08-g30'
path5 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p85-g30'
path6 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p06-g10'
path7 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p08-g10'
path8 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p85-g10'
path9 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p06-g13'
path10 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p08-g13'
path11 = '/home/froehlich/bnelearn/experiments/decrease/PSOLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/07-07-p85-g13'
paths = [path1, path2, path3, path4, path5, path6, path7, path8, path9, path10, path11]
keys = ['T_g = exp(-3/64), T_p = exp(-0.2)', 'T_g = exp(-3/64), T_p = exp(-0.4)', 'T_g = exp(-3/64), T_p = exp(-0.6)', 'T_g = exp(-3/64), T_p = exp(-0.8)', 'T_g = exp(-3/64), T_p = exp(-0.85)', 'T_g = exp(-1.0), T_p = exp(-0.6)', 'T_g = exp(-1.0), T_p = exp(-0.8)', 'T_g = exp(-1.0), T_p = exp(-0.85)', 'T_g = exp(-1.3), T_p = exp(-0.6)', 'T_g = exp(-1.3), T_p = exp(-0.8)', 'T_g = exp(-1.3), T_p = exp(-0.85)']
criteria = ['eval/epsilon_relative']

df = return_df(return_path, return_name, paths, keys, criteria)
df

,,mean,std
experiment,criteria,,
"T_g = exp(-3/64), T_p = exp(-0.2)",epsilon_relative,0.0008,0.0009
"T_g = exp(-3/64), T_p = exp(-0.4)",epsilon_relative,0.0008,0.0007
"T_g = exp(-3/64), T_p = exp(-0.6)",epsilon_relative,0.0005,0.0003
"T_g = exp(-3/64), T_p = exp(-0.8)",epsilon_relative,0.0010,0.0009
"T_g = exp(-3/64), T_p = exp(-0.85)",epsilon_relative,0.0007,0.0006
"T_g = exp(-1.0), T_p = exp(-0.6)",epsilon_relative,0.0014,0.0024
"T_g = exp(-1.0), T_p = exp(-0.8)",epsilon_relative,0.0011,0.0013
"T_g = exp(-1.0), T_p = exp(-0.85)",epsilon_relative,0.0006,0.0004
"T_g = exp(-1.3), T_p = exp(-0.6)",epsilon_relative,0.0016,0.0024
